In [4]:
import cv2
import mediapipe as mp
import numpy as np
import math
import datetime
import autopy # controlling in mause and key board
import joblib
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import os 
import statistics
import pickle
from random import random
from time import sleep
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import wait

#### Calculate Angle Between Three Point (1.shoulder,2.elbow,3.wrist)

In [5]:
def calculate_angle(SHOULDER,ELBOW,WRIST):
    """
    _ calculate_angle:Function to calculate angle between three point (1.shoulder,2.elbow,3.wrist)
    _ shoulder:pose landmark number 11 and 12 containe X_position and Y_position
    _ elbow:pose landmark number 13 and 14 containe X_position and Y_position
    _ wrist:pose landmark number 15 and 16 containe X_position and Y_position
    """
    SHOULDER = np.array(SHOULDER) # First
    ELBOW = np.array(ELBOW) # Mid
    WRIST = np.array(WRIST) # End
    
    radians = np.arctan2(WRIST[1]-ELBOW[1],WRIST[0]-ELBOW[0])-np.arctan2(SHOULDER[1]-ELBOW[1],SHOULDER[0]-ELBOW[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180:
        angle = 360-angle
    return angle

#### Calculate Distances Between Point in Curent Frame and Point in Previose Frame

In [6]:
def calculate_distances(delta_t,landmark_1,landmark_2):
    """
    _ calculate_distances:Function to calculate distances for specific landmark (shoulder,elbow,wrist,hip,knee,ankle "_left&right_")
    between point in curent frame and point in previose frame
    _ index:number of curent frame
    _ fps:number of frame per seconde
    _ landmark_1:pose landmark for curent frame
    _ landmark_2:pose landmark for previose frame
    """
    my_list = []
    for ind in range(len(landmark_1)):
        if ((ind>=0)and(ind<=10))or((ind>=17)and(ind<=22))or((ind>=29)and(ind<=32)):
            continue
        p1 = [landmark_1[ind].x,landmark_1[ind].y]
        p2 = [landmark_2[ind].x,landmark_2[ind].y]
        my_list.append(np.array([[p1[0]],[p1[1]],[(p1[0]-p2[0])/delta_t],[(p1[1]-p2[1])/delta_t]]))
    return my_list

#### Kalman Filter Equation Prediction Update 

In [7]:
def Prediction_update(X,A,P,Q):
    """
    _ Prediction_update:Function to Applying Prediction Equation
    _ X:State Matrix [X_position,Y_position,X_velocity,Y_velocity]
    _ A:State Transition
    _ P:Process Covariance Matrix
    _ Q:Erorr Terms
    """
    X = A.dot(X)
    TEPM = A.dot(P)
    P = TEPM.dot(A.T)+Q
    return X,P

##### Kalman Filter Equation Measurement Update

In [8]:
def Measurement_update(Z,X,P,H,R):
    """
    _ Measurement_update:Function to Applying Measurement Equation
    _ X:State Matrix [X_position,Y_position,X_velocity,Y_velocity]
    _ P:Process Covariance Matrix
    _ H:unit matrix
    _ R:Erorr Terms of the Measurement
    """
    I = np.array([[1,0, 0,0],
                  [0, 1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]])
    
    T1 = P.dot(H.T)
    T2 = H.dot(P)
    T3 = T2.dot(H.T)
    T3 = T3 + R
    K = T1.dot(np.linalg.inv(T3))
    
    T1 = H.dot(X)
    T2 = Z - T1
    T3 = K.dot(T2)
    X = X + T3
    
    T1 = K.dot(H)
    T2 = I - T1
    P = T2.dot(P)

    return X,P,K

#### get all video in any root path

In [9]:
def get_videos(path_folder):
    """
    _ get_videos:Function to get all videos in root path
    _ input:(path_folder) the root path for all dirctory that contant video
    _ output:(folders) all dirctory in root path (all_video_paths) all paths video
    """
    folders = os.listdir(path_folder)
    all_video_paths = []
    for folder in folders:
        curr_dataset_path = os.path.join(data_path, folder)
        videos = os.listdir(curr_dataset_path)
        for video_p in videos:
            all_video_paths.append(os.path.join(curr_dataset_path, video_p))
    return folders , all_video_paths

#### Calculate Average for List or Array of Numbers

In [10]:
def Average(lst):
    """
    _Average:Function to calculate mean for list or array
    _input:List or array of numbers
    _output:the mean for input numbre
    """
    return sum(lst) / len(lst)

#### Save any Object We Need 

In [11]:
def save_object(obj, filename):
    """
    _save_object:Function to save any object
    _input:(obj) the object that we want save , (filename) Memorization path 
    """
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

#### Save any Object

In [12]:
def load_object(filename):
    """
    _load_object:Function to load any object
    _input:object path 
    _output:the object
    """
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

#### calculate angles between shoulder and elbow and wrist point for left and right hand

In [13]:
def calculate_left_right_arm_angle(landmarks,mp_pose):
    """
    _ calculate_left_right_arm_angle:Function to calculate angle between three points from arm 
    _ input: (landmarks) all pose points , (mp_pose) contante all landmark name
    _ output: left and right angle for tow arm
    """
    # Get coordinates for left points
    LEFTshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    LEFTelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    LEFTwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

    # calculate LEFT angle
    LEFTangle = calculate_angle(LEFTshoulder,LEFTelbow,LEFTwrist)

    # Get coordinates for right points
    RIGHTshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    RIGHTelbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    RIGHTwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

    # calculate RIGHT angle
    RIGHTangle = calculate_angle(RIGHTshoulder,RIGHTelbow,RIGHTwrist)
    return LEFTangle,RIGHTangle

#### Check if ram is moving or not

In [14]:
def check_hand_moving(pre_angle,cur_angle):
    """
    _ check_hand_moving: Function to check if arm moving or not 
    _ input: (pre_angle) angle from previose frame (cur_angle) angle from curent frame
    _ output: (True) if arm moving (False) if arm not moving 
    """
    angle = cur_angle-pre_angle
    stage=False
    if angle>5:
        stage = True
    return stage

#### give the results of the calculation

In [15]:
def hand_get_decistion(l_is_moved,r_is_moved):
    """
    _ hand_get_decistion: get decis
    _ input: ("l_is_moved") true if left arm is moved or false if not , ("r_is_moved") true if right arm is moved or false if not
    _ output: (Moving Hand , 1) or (Rest Hand 0)
    """
    if l_is_moved or r_is_moved:
        return "Moving Hand",1
    return "Rest Hand",0

In [16]:
def Drawing_utils(image,results,mp_pose):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=4,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=4))

In [17]:
def creat_kalman_parameters(delta_t):
    A = np.array([[1,0, delta_t,0],[0, 1,0,delta_t],[0,0,1,0],[0,0,0,1]]) # STATE MATRIC (The relationship between system variables in the equations of motion)
    P = np.eye(4)*10 # Guess Kalmen , Uncertainty Matrix , The relationship of state variables , external forces
    H = np.eye(4) # Measurement matrix is used to calculate Calmen error
    R = np.eye(4)*0.05 # Noise added for update
    Q =np.eye(4)*0.1 # Noise added for measurement
    X = []
    for i in range(12):
        X.append(np.array([[0],[0],[0],[0]]))
    return A,P,H,R,Q,X

#### Download the video, perform the required calculations, and then save result

In [39]:
def hand_pipeline(path_A):
    decistion = ""
    path_A_ = path_A.split("\\")
    key = path_A_[-1].split(".")[-2].split("-")[-1]+" "+path_A_[-2].split("-")[-1]
    FrameLable[key] = []
    points_in_frames = []
    my_list = []
    index = 0 
    mp_pose = mp.solutions.pose 
    cap = cv2.VideoCapture(path_A)
    # We need to check if camera
    # is opened previously or not
    if (cap.isOpened() == False): 
        print("Error reading video file")
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)
    # count the number of frames
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # calculate duration of the video
    seconds = round(frames / fps)
    video_time = datetime.timedelta(seconds=seconds)
    delta_t = 1/fps
    A,P,H,R,Q,X = creat_kalman_parameters(delta_t)
    # Pose : is the algorithem that will be do detection for 33 pointes
    ## setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if (not ret) or (cv2.waitKey(10)==ord('q')):
#                 print("Ignoring empty frame.")
                # If loading a video, use 'break' instead of 'continue'.
                break
            # Recolor image to RGB
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            # Make detection
            results = pose.process(image)
            # Recolor to BGR
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
            # Extract lnadmarks
            try:
                landmarks = results.pose_landmarks.landmark
                points_in_frames.append(landmarks)
                LEFTangle , RIGHTangle = calculate_left_right_arm_angle(landmarks,mp_pose)
                freeLEFT.append(LEFTangle)
                freeRIGHT.append(RIGHTangle)
                if len(freeLEFT)>1:
                    l_is_moved = check_hand_moving(freeLEFT[-2],freeLEFT[-1])
                else:
                    l_is_moved = False
                if len(freeRIGHT)>1:
                    r_is_moved =check_hand_moving(freeRIGHT[-2],freeRIGHT[-1])
                else:
                    r_is_moved = False
                decistion,Ndecistion = hand_get_decistion(l_is_moved,r_is_moved)
                FrameLable[key].append(Ndecistion)
                
                if index!=0 and (index%fps==0):
                    my_list = calculate_distances(delta_t,points_in_frames[index],points_in_frames[index-1])
                    if index==15:
                        X = my_list
                    for i in range(len(my_list)):
                        Z = my_list[i]
                        X[i] , P = Prediction_update(X[i],A,P,Q)
                        X[i] , P , K = Measurement_update(Z,X[i],P,H,R)
                index+=1
            except:
                pass
            cv2.putText(image,decistion,(80,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
#             Drawing_utils(image,results,mp_pose)
            
            cv2.imshow('Body Movement Taske',image)
            if cv2.waitKey(10)==ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

In [40]:
# data_path = "D:\DataSet"
# folders ,videos = get_videos(data_path)
freeRIGHT = []
freeLEFT = []
FrameLable = {}
# executor = ThreadPoolExecutor(max_workers=10)
# executor.map(pipeline, videos)
# executor.shutdown()
# save_object(FrameLable,"FrameLable")

In [42]:
path_A = "D:\DataSet\Afghanistan And Iraq_ Were they _Just Wars__ _ Eloise Burkey _ TEDxFrancisHollandSchoolSloaneSquare(360P).mp4"
# path_A = "D:\DataSet\Procrastination _ Jacqueline Haas _ TEDxFrancisHollandSchoolSloaneSquare(720P_HD).mp4"
path_A = "D:\DataSet\Afghanistan And Iraq_ Were they _Just Wars__ _ Eloise Burkey _ TEDxFrancisHollandSchoolSloaneSquare.mp4"

hand_pipeline(path_A)

In [21]:
def dist(p1,p2):
    return math.sqrt((p2[0]-p1[0])**2+(p2[1]-p1[1])**2)

In [25]:
def check_body_moving(i,head_points,elbo_points):
    if (head_points[-1]>elbo_points[-1][1]) and (head_points[-1]<elbo_points[-1][0]):
        return False
    return True

In [26]:
def pipeline(path_A):
    decistion=""
    temp = True
    head_points = []
    elbo_points = []
    path_A_ = path_A.split("\\")
    key = path_A_[-1].split(".")[-2].split("-")[-1]+" "+path_A_[-2].split("-")[-1]
    points_in_frames = []
    my_list = []
    index = 0 
    mp_pose = mp.solutions.pose 
    cap = cv2.VideoCapture(path_A)
    if (cap.isOpened() == False): 
        print("Error reading video file")
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    seconds = round(frames / fps)
    video_time = datetime.timedelta(seconds=seconds)
    delta_t = 1/fps
    A,P,H,R,Q,X = creat_kalman_parameters(delta_t)
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if (not ret) or (cv2.waitKey(10)==ord('q')):
                break
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
            try:
                landmarks = results.pose_landmarks.landmark
                points_in_frames.append(landmarks)
                
                NOSE = landmarks[mp_pose.PoseLandmark.NOSE.value].x#,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
                LEFT_SHOULDER = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x#,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                RIGHT_SHOULDER = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x#,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                if NOSE and temp:
                    temp=False
                    head_points.append(NOSE)
                    elbo_points.append([LEFT_SHOULDER,RIGHT_SHOULDER])
                if len(head_points)>=1:
                    head_points.append(NOSE)
                    if check_body_moving(index,head_points,elbo_points):
                        elbo_points.append([LEFT_SHOULDER,RIGHT_SHOULDER])
                        decistion = "Moving Body"
                    else:
                        decistion = "Rest Body"                
                if index!=0 and (index%fps==0):
                    my_list = calculate_distances(delta_t,points_in_frames[index],points_in_frames[index-1])
                    if index==15:
                        X = my_list
                    for i in range(len(my_list)):
                        Z = my_list[i]
                        X[i] , P = Prediction_update(X[i],A,P,Q)
                        X[i] , P , K = Measurement_update(Z,X[i],P,H,R)
            except:
                pass
            cv2.putText(image,decistion,(80,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
            Drawing_utils(image,results,mp_pose)
            
            cv2.imshow('Body Movement Taske',image)
            if cv2.waitKey(10)==ord('q'):
                break
            index+=1     
    cap.release()
    cv2.destroyAllWindows()

In [36]:
path_A = "D:\DataSet\Afghanistan And Iraq_ Were they _Just Wars__ _ Eloise Burkey _ TEDxFrancisHollandSchoolSloaneSquare.mp4"
# path_A = "D:\DataSet\Afghanistan And Iraq_ Were they _Just Wars__ _ Eloise Burkey _ TEDxFrancisHollandSchoolSloaneSquare(360P).mp4"
# path_A = "D:\DataSet\Procrastination _ Jacqueline Haas _ TEDxFrancisHollandSchoolSloaneSquare(720P_HD).mp4"
# path_A = "D:\DataSet\Presenter No1-Asylah\C_presenter No1-groupA.mp4"
pipeline(path_A)

=================0===================
head :  0.5148856043815613
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================1===================
head :  0.5153971910476685
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================2===================
head :  0.5157603621482849
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================3===================
head :  0.5163964629173279
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================4===================
head :  0.5171480774879456
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================5===================
head :  0.5178436040878296
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================6===================
head :  0.5185400247573853
left :  0.46625208854675293
right :  0.5808897018432617
Rest Body
=================7===================
head :  0.5198525190353394
left :  0.46625208

=================64===================
head :  0.4085307717323303
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================65===================
head :  0.4069124162197113
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================66===================
head :  0.40515297651290894
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================67===================
head :  0.4039972722530365
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================68===================
head :  0.4036427140235901
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================69===================
head :  0.4028424322605133
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================70===================
head :  0.40159305930137634
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================71===================
head :  0.4009057581424713
left :  0.38097

=================127===================
head :  0.45771390199661255
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================128===================
head :  0.4586494565010071
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================129===================
head :  0.45913904905319214
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================130===================
head :  0.4596793055534363
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================131===================
head :  0.46044406294822693
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================132===================
head :  0.4612044394016266
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================133===================
head :  0.4615744948387146
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================134===================
head :  0.4617311358451843
left :

=================192===================
head :  0.4621909558773041
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================193===================
head :  0.4633522033691406
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================194===================
head :  0.4642193913459778
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================195===================
head :  0.4650261402130127
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================196===================
head :  0.4661934971809387
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================197===================
head :  0.46703752875328064
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================198===================
head :  0.46811461448669434
left :  0.3809714913368225
right :  0.4866264760494232
Rest Body
=================199===================
head :  0.4688010811805725
left : 

=================255===================
head :  0.46415820717811584
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================256===================
head :  0.46085190773010254
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================257===================
head :  0.4579339623451233
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================258===================
head :  0.4544812738895416
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================259===================
head :  0.4521714746952057
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================260===================
head :  0.4499465823173523
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================261===================
head :  0.4487209916114807
left :  0.4304382801055908
right :  0.5499639511108398
Rest Body
=================262===================
head :  0.4461590647697449
left : 

...

=================572===================
head :  0.4832107126712799
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================573===================
head :  0.4831960201263428
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================574===================
head :  0.4831555485725403
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================575===================
head :  0.4829568862915039
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================576===================
head :  0.4829394817352295
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================577===================
head :  0.48292455077171326
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================578===================
head :  0.48290809988975525
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================579===================
head :  0.48287075757980347

=================634===================
head :  0.4626704752445221
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================635===================
head :  0.460993230342865
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================636===================
head :  0.4600839614868164
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================637===================
head :  0.4592452049255371
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================638===================
head :  0.4587094187736511
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================639===================
head :  0.4581802785396576
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================640===================
head :  0.4569459557533264
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================641===================
head :  0.45591363310813904
le

=================696===================
head :  0.4791082739830017
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================697===================
head :  0.4804508686065674
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================698===================
head :  0.4826374053955078
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================699===================
head :  0.4841694235801697
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================700===================
head :  0.4859530031681061
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================701===================
head :  0.4880867600440979
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================702===================
head :  0.49008291959762573
left :  0.39787545800209045
right :  0.5063689351081848
Rest Body
=================703===================
head :  0.49194827675819397


=================758===================
head :  0.4427497982978821
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================759===================
head :  0.4422624707221985
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================760===================
head :  0.4419930875301361
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================761===================
head :  0.4417501986026764
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================762===================
head :  0.4418559670448303
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================763===================
head :  0.44183939695358276
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================764===================
head :  0.44202908873558044
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================765===================
head :  0.4421599805355072
left : 

=================821===================
head :  0.4837729334831238
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================822===================
head :  0.4836723804473877
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================823===================
head :  0.483590692281723
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================824===================
head :  0.4835757315158844
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================825===================
head :  0.4835611879825592
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================826===================
head :  0.4836013913154602
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================827===================
head :  0.48388028144836426
left :  0.41971999406814575
right :  0.522911787033081
Rest Body
=================828===================
head :  0.48433923721313477
left :  

In [28]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    cv2.imshow('Frame',frame)
    print(frame.shape) #prints image dimension
    print(cv2.getWindowImageRect('Frame'))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 57, 1280, 720)
(480, 640, 3)
(147, 

In [34]:
# save_object(FrameLable,"FrameLable")
# FrameLable = load_object('FrameLable')
# len(FrameLable)

74

In [19]:
# for j,i in enumerate(FrameLable):
#     print(j,"_ " +i+"=> "+"|number of frame : ",len(FrameLable[i]),"|number of positive frame => ",sum(FrameLable[i]),"|number of negative frame => ",len(FrameLable[i])-sum(FrameLable[i]))

In [21]:
# data_path = "D:\DataSet"
# folders ,videos = get_videos(data_path)
# xs_velocity_for_all_videos = []
# xb_velocity_for_all_videos = []
# ys_velocity_for_all_videos = []
# yb_velocity_for_all_videos = []
freeRIGHT = []
freeLEFT = []
# for path_A in  videos[:4]:
#     cc=0
#     xs_velocity_for_all_frams = []
#     xb_velocity_for_all_frams = []
#     ys_velocity_for_all_frams = []
#     yb_velocity_for_all_frams = []
path_A = "D:\DataSet\Afghanistan And Iraq_ Were they _Just Wars__ _ Eloise Burkey _ TEDxFrancisHollandSchoolSloaneSquare(360P).mp4"
path_A1 = path_A.split('.')[0]+'.avi'
#     counter = 0
decistion = ""
points_in_frames = []
my_list = []
index = 0 
mp_pose = mp.solutions.pose 
cap = cv2.VideoCapture(path_A)
# We need to check if camera
# is opened previously or not
if (cap.isOpened() == False): 
    print("Error reading video file")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
#     result = cv2.VideoWriter(path_A1,cv2.VideoWriter_fourcc(*'MJPG'),10, size)
# count the number of frames
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
# calculate duration of the video
seconds = round(frames / fps)
video_time = datetime.timedelta(seconds=seconds)
delta_t = 1/fps
A,P,H,R,Q,X = creat_kalman_parameters(delta_t)
#     xs = [[],[],[],[],[],[],[],[],[],[],[],[]]
#     ys = [[],[],[],[],[],[],[],[],[],[],[],[]]
#     xb = [[],[],[],[],[],[],[],[],[],[],[],[]]
#     yb = [[],[],[],[],[],[],[],[],[],[],[],[]]
# Pose : is the algorithem that will be do detection for 33 pointes
## setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if (not ret) or (cv2.waitKey(10)==ord('q')):
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break
        # Recolor image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        # Make detection
        results = pose.process(image)
        # Recolor to BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        # Extract lnadmarks
        try:
            landmarks = results.pose_landmarks.landmark
            points_in_frames.append(landmarks)
            LEFTangle , RIGHTangle = calculate_left_right_arm_angle(landmarks,mp_pose)
            freeLEFT.append(LEFTangle)
            freeRIGHT.append(RIGHTangle)
            if len(freeLEFT)>1:
                l_is_moved = cheak_hand_moving(freeLEFT[-2],freeLEFT[-1])
            else:
                l_is_moved = False
            if len(freeRIGHT)>1:
                r_is_moved =cheak_hand_moving(freeRIGHT[-2],freeRIGHT[-1])
            else:
                r_is_moved = False
            decistion = hand_get_decistion(l_is_moved,r_is_moved)
            if index!=0 and (index%fps==0):
                my_list = calculate_distances(delta_t,points_in_frames[index],points_in_frames[index-1])
                if index==15:
                    X = my_list
                for i in range(len(my_list)):
                    Z = my_list[i]
                    X[i] , P = Prediction_update(X[i],A,P,Q)
                    X[i] , P , K = Measurement_update(Z,X[i],P,H,R)
#                         xs[i].append(X[i][2][0])
#                         ys[i].append(X[i][3][0])
#                         if threshold[i]<=abs(X[i][2][0]):
#                             cc+=1
#                             print(str(i)+" : movment")
#                         print("Z"+str(i)+" : \n",Z)
#                         print("X"+str(i)+" : \n",X[i])
#                         print("P"+str(i)+" : \n",P)
#                     print("\n\n")
            if len(X)>0:
#                 for i in range(len(my_list)):
#                     width=1100
#                     if i%2==0 :
#                         width=1300
#                     if i==2 or i==3:
#                         cv2.putText(image,str(str(round(my_list[i][2][0],4))+" , "+str(round(my_list[i][3][0],4))+"   "+str(int(LEFTangle))),(int(my_list[i][0][0]*width),int(my_list[i][1][0]*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
#                     else:
#                         cv2.putText(image,str(str(round(my_list[i][2][0], 4))+" , "+str(round(my_list[i][3][0], 4))),(int(my_list[i][0][0]*width),int(my_list[i][1][0]*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[0][2][0], 4))+" , "+str(round(X[0][3][0], 4))),(int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[1][2][0],4))+" , "+str(round(X[1][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[2][2][0],4))+" , "+str(round(X[2][3][0],4))+"   "+str(int(LEFTangle))),(int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[3][2][0],4))+" , "+str(round(X[3][3][0],4))+"   "+str(int(RIGHTangle))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[4][2][0],4))+" , "+str(round(X[4][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[5][2][0],4))+" , "+str(round(X[5][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[6][2][0],4))+" , "+str(round(X[6][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[7][2][0],4))+" , "+str(round(X[7][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[8][2][0],4))+" , "+str(round(X[8][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[9][2][0],4))+" , "+str(round(X[9][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[10][2][0],4))+" , "+str(round(X[10][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x*1300),int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(str(round(X[11][2][0],4))+" , "+str(round(X[11][3][0],4))),(int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x*1100),int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y*700)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            index+=1
        except:
            pass
        # Render culr counter
        # setup status box
#             cv2.rectangle(image,(0,0),(235,73),(245,117,16),-1)
        # Rep data
#             cv2.putText(image,'PEPS',(15,12),
#                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
#             cv2.putText(image,str(counter),
#                        (10,60),
#                        cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255))
#             cv2.putText(image,'STAG',(75,12),
#                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0))
        cv2.putText(image,decistion,(80,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
        Drawing_utils(image,results,mp_pose)
#             result.write(image)
        cv2.imshow('Body Movement Taske',image)
        if cv2.waitKey(10)==ord('q'):
            break
#         for xss in xs:
#             if xss:
#                 x_res =  [abs(ele) for ele in xss]
#                 x_min_res = min(x_res)
#                 x_max_res = max(x_res)
#                 xs_velocity_for_all_frams.append(x_min_res)
#                 xb_velocity_for_all_frams.append(x_max_res)
#         for yss in ys:
#             if yss:
#                 y_res =  [abs(ele) for ele in yss]
#                 y_min_res = min(x_res)
#                 y_max_res = max(x_res)
#                 ys_velocity_for_all_frams.append(y_min_res)
#                 yb_velocity_for_all_frams.append(y_max_res)
#         xs_velocity_for_all_videos.append(xs_velocity_for_all_frams)
#         xb_velocity_for_all_videos.append(xb_velocity_for_all_frams)
#         ys_velocity_for_all_videos.append(ys_velocity_for_all_frams)
#         yb_velocity_for_all_videos.append(yb_velocity_for_all_frams)
#     print(cc)
cap.release()
#     result.release()
cv2.destroyAllWindows()  

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - Can't convert object to 'str' for 'text'
>  - Can't convert object to 'str' for 'text'


In [15]:
# save_object(xs_velocity_for_all_videos,"xs_velocity_for_all_videos")
# save_object(ys_velocity_for_all_videos,"ys_velocity_for_all_videos")
# save_object(xb_velocity_for_all_videos,"xb_velocity_for_all_videos")
# save_object(yb_velocity_for_all_videos,"yb_velocity_for_all_videos")

In [14]:
xs_velocity_for_all_videos = np.array(load_object("xs_velocity_for_all_videos"))
ys_velocity_for_all_videos = np.array(load_object("ys_velocity_for_all_videos"))
xb_velocity_for_all_videos = np.array(load_object("xb_velocity_for_all_videos"))
yb_velocity_for_all_videos = np.array(load_object("yb_velocity_for_all_videos"))

In [15]:
xs_velocity_for_all_videos[0]

array([6.02633538e-07, 5.86057451e-06, 2.68474684e-05, 2.69540077e-06,
       7.83645995e-06, 1.06474067e-04, 2.84171326e-06, 7.12373552e-07,
       5.45661354e-05, 3.64366872e-05, 4.36592409e-06, 6.26613320e-06])

In [16]:
xb_velocity_for_all_videos[0]

array([0.00764118, 0.01121931, 0.01535066, 0.03498155, 0.07550554,
       0.06640266, 0.01289561, 0.01194002, 0.12288177, 0.14963705,
       0.10753535, 0.03212713])

In [17]:
xb_velocity_for_all_videos[0]-xs_velocity_for_all_videos[0]

array([0.00764058, 0.01121345, 0.01532382, 0.03497885, 0.07549771,
       0.06629618, 0.01289277, 0.01193931, 0.12282721, 0.14960061,
       0.10753098, 0.03212087])

In [18]:
xs_velocity_for_all_videos

array([[6.02633538e-07, 5.86057451e-06, 2.68474684e-05, 2.69540077e-06,
        7.83645995e-06, 1.06474067e-04, 2.84171326e-06, 7.12373552e-07,
        5.45661354e-05, 3.64366872e-05, 4.36592409e-06, 6.26613320e-06],
       [4.97781295e-06, 8.91365236e-06, 2.34565570e-06, 2.02207076e-05,
        1.14361110e-04, 9.50309937e-05, 6.03020504e-05, 4.27305742e-05,
        2.74387659e-05, 1.17887104e-04, 2.13402096e-04, 9.01185228e-05]])

In [19]:
for i in range(len(xs_velocity_for_all_videos[0])):
    for j in xs_velocity_for_all_videos:
        j[i]

In [20]:
xb_velocity_for_all_videos[1]

array([0.05399833, 0.04608578, 0.06320592, 0.09593811, 0.61692073,
       0.24760956, 0.04581504, 0.0550016 , 0.06876897, 0.11519189,
       0.11678388, 0.09608118])

In [21]:
xb_velocity_for_all_videos[1]-xs_velocity_for_all_videos[1]

array([0.05399336, 0.04607686, 0.06320357, 0.09591788, 0.61680637,
       0.24751453, 0.04575473, 0.05495887, 0.06874153, 0.115074  ,
       0.11657048, 0.09599106])

In [22]:
threshold = ((xb_velocity_for_all_videos[1]+xs_velocity_for_all_videos[0])/2)

In [23]:
threshold

array([0.02699947, 0.02304582, 0.03161638, 0.0479704 , 0.30846428,
       0.12385802, 0.02290894, 0.02750116, 0.03441177, 0.05761416,
       0.05839412, 0.04804372])